In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
from Pytorch_VAE_LSTM import LSTMVAE
from music_dataset import TorchMusicDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
import numpy as np

import scipy.io.wavfile as wavfile

In [2]:
epochs = 400
batch_size = 4
hidden_dim = 100
latent_dim = 100
n_seconds = 10

In [3]:
evolution_folder = "evolution"

In [4]:
if not os.path.exists(evolution_folder):
    os.mkdir(evolution_folder)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
dataset = TorchMusicDataset("../cleaned_data", n_seconds)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
n_timesteps, n_features  = dataset.__getitem__(1).shape[:2]
n_timesteps, n_features

(44100, 11)

In [7]:
dataset.__getitem__(1).shape

torch.Size([44100, 11])

In [ ]:
vae = LSTMVAE(n_timesteps, n_features, hidden_dim, latent_dim, device).to(device)
vae.optimizer = optim.Adam(vae.parameters())

In [ ]:
%%time
epoch_losses = []
for epoch in range(epochs):
    epoch_loss = 0
    n_steps = 0
    for batch_x in dataloader:
        batch_loss = vae.training_step(batch_x.to(device))
        epoch_loss += batch_loss
        n_steps += 1
    epoch_loss = epoch_loss/n_steps
    epoch_losses.append(epoch_loss)
    print(f"Epoch {epoch} loss: {epoch_loss}")
    if (epoch % 10) == 0:
        torch.save(vae.state_dict(), "model.torch")
        with open("losses.json", "w") as f:
            json.dump(epoch_losses, f)

In [ ]:
torch.save(vae.state_dict(), "model.torch")

In [ ]:
with open("losses.json", "w") as f:
    json.dump(epoch_losses, f)

In [ ]:
print("END")